<a href="https://colab.research.google.com/github/orcorsetti/labo2025v/blob/main/src/ensembles/420_ArbolesAzarosos_Bayesian.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 4. Ensembles de Arboles de Decision

Un arbol de decisión es un modelo débil, el aumento del poder predictivo proviene al ensamblar varios arboles de decisión.
<br> Si promedio n arboles identicos, el resultados es exactamente el mismo que utilizar un solo arbol, necesito PERTURBAR cada arbol para disponer de variablidad

la variabilidad provendrá de estas fuentes:


*   Perturbar el dataset
*   Perturbar el algoritmo del arbol
*   Perturbar el dataset y el algoritmo del arbol al mismo tiempo

Se verán estos tres algoritmos


*   Arboles Azarosos
*   Random Forest
*   Gradient Boosting of Decision Trees

#### 4.01 Seteo del ambiente en Google Colab

Esta parte se debe correr con el runtime en Python3
<br>Ir al menu, Runtime -> Change Runtime Type -> Runtime type ->  **Python 3**

Conectar la virtual machine donde esta corriendo Google Colab con el  Google Drive, para poder tener persistencia de archivos

In [1]:
# primero establecer el Runtime de Python 3
from google.colab import drive
drive.mount('/content/.drive')

Drive already mounted at /content/.drive; to attempt to forcibly remount, call drive.mount("/content/.drive", force_remount=True).


Para correr la siguiente celda es fundamental en Arranque en Frio haber copiado el archivo kaggle.json al Google Drive, en la carpeta indicada en el instructivo

<br>los siguientes comando estan en shell script de Linux
*   Crear las carpetas en el Google Drive
*   "instalar" el archivo kaggle.json desde el Google Drive a la virtual machine para que pueda ser utilizado por la libreria  kaggle de Python
*   Bajar el  **dataset_pequeno**  al  Google Drive  y tambien al disco local de la virtual machine que esta corriendo Google Colab



In [2]:
%%shell

mkdir -p "/content/.drive/My Drive/labo1"
mkdir -p "/content/buckets"
ln -s "/content/.drive/My Drive/labo1" /content/buckets/b1

mkdir -p ~/.kaggle
cp /content/buckets/b1/kaggle/kaggle.json  ~/.kaggle
chmod 600 ~/.kaggle/kaggle.json


mkdir -p /content/buckets/b1/exp
mkdir -p /content/buckets/b1/datasets
mkdir -p /content/datasets



archivo_origen="https://storage.googleapis.com/open-courses/austral2025-af91/dataset_pequeno.csv"
archivo_destino="/content/datasets/dataset_pequeno.csv"
archivo_destino_bucket="/content/buckets/b1/datasets/dataset_pequeno.csv"

if ! test -f $archivo_destino_bucket; then
  wget  $archivo_origen  -O $archivo_destino_bucket
fi


if ! test -f $archivo_destino; then
  cp  $archivo_destino_bucket  $archivo_destino
fi



---



## 4.02 Arboles Azarosos

Arboles Azarosos es el nombre de un algoritmo trivial (por favor NO confundir con Random Forest)
Qué tipo de perturbaciones se realizan en Arboles Azarosos
* Se perturba el dataset
* No se perturba el algoritmo, es siempre rpart original

Cada  arbolito de  Arboles Azarosos se entrena sobre un dataset perturbado,  que tiene exactamente la misma cantidad de registros pero solo un subconjunto de los atributos (campos)  del dataset, tomados al azar, de los originales.
<br> En esta primera corrida, se construira cada arbol en un dataset utilizando el 50% de los campos

Esta parte se debe correr con el runtime en lenguaje **R** Ir al menu, Runtime -> Change Runtime Type -> Runtime type -> R

limpio el ambiente de R

In [357]:
format(Sys.time(), "%a %b %d %X %Y")

[1] "Wed Oct 29 12:43:37 AM 2025"

In [358]:
# limpio la memoria
rm(list=ls(all.names=TRUE)) # remove all objects
gc(full=TRUE, verbose=FALSE) # garbage collection

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,2672748,142.8,4773622,255.0,4773622,255.0
Vcells,4856309,37.1,104883703,800.2,195324188,1490.3


In [359]:
# cargo las librerias que necesito
require("data.table")
require("rpart")
require("parallel")

In [360]:
if( ! require("rlist") ) install.packages("rlist")
require("rlist")

In [361]:
# paquete necesarios para la Bayesian Optimization
if( !require("DiceKriging") ) install.packages("DiceKriging")
require("DiceKriging")

In [362]:
# paquete necesarios para la Bayesian Optimization
if( !require("mlrMBO") ) install.packages("mlrMBO")
require("mlrMBO")

Aqui debe cargar SU semilla primigenia

In [404]:
PARAM <- list()
PARAM$semilla_primigenia <- 100183

# parametros  arbol
# entreno cada arbol con solo 50% de las variables variables
#  por ahora, es fijo
PARAM$feature_fraction <- 0.5

PARAM$rpart$cp <- -1
PARAM$rpart$minsplit <- 50
PARAM$rpart$minbucket <- 20
PARAM$rpart$maxdepth <- 6

# voy a generar 512 arboles,
#  a mas arboles mas tiempo de proceso y MEJOR MODELO,
#  pero ganancias marginales
PARAM$num_trees_max <- 2

In [385]:
# carpeta de trabajo
setwd("/content/buckets/b1/exp")
experimento <- "exp4022"
dir.create(experimento, showWarnings=FALSE)
setwd( paste0("/content/buckets/b1/exp/", experimento ))

In [386]:
# lectura del dataset
dataset <- fread("/content/datasets/dataset_pequeno.csv")

In [387]:
# defino los dataset de entrenamiento y aplicacion
dtrain <- dataset[foto_mes == 202107]
dfuture <- dataset[foto_mes == 202109]


In [388]:
# Establezco cuales son los campos que puedo usar para la prediccion
# el copy() es por la Lazy Evaluation
campos_buenos <- copy(setdiff(colnames(dtrain), c("clase_ternaria")))

In [389]:
set.seed(PARAM$semilla_primigenia) # Establezco la semilla aleatoria

In [390]:
particionar <- function(data, division, agrupa = "", campo = "fold",
                        start = 1, seed = NA) {
  if (!is.na(seed)) set.seed(seed)

  bloque <- unlist(mapply(
    function(x, y) {
      rep(y, x)
    }, division,
    seq(from= start, length.out= length(division))
  ))

  data[, (campo) := sample(rep(bloque, ceiling(.N / length(bloque))))[1:.N],
    by= agrupa
  ]
}

In [411]:
EstimarGanancia = function(fold_test, x) {

    tb_prediccion <- dfuture[, list(numero_de_cliente)]
    # aqui se va acumulando la probabilidad del ensemble
    tb_prediccion[, prob_acumulada := 0]

    for (arbolito in seq(PARAM$num_trees_max) ) {
    message( arbolito, " ")
    qty_campos_a_utilizar <- as.integer(length(campos_buenos)
      * PARAM$feature_fraction)

    # elijo los campos al azar
    campos_random <- sample(campos_buenos, qty_campos_a_utilizar)

    # paso de un vector a un string con los elementos
    # separados por un signo de "+"
    # este hace falta para la formula
    campos_random <- paste(campos_random, collapse= " + ")

    # armo la formula para rpart
    formulita <- paste0("clase_ternaria ~ ", campos_random)


    # genero el arbol de decision
    modelo <- rpart(formulita,
      data= dataset[fold != fold_test, ],
      xval= 0,
      control= x
    )

    # aplico el modelo a los datos que no tienen clase
    prediccion <- predict(modelo, dataset[fold == fold_test, ], type= "prob")

    tb_prediccion[, prob_acumulada := prob_acumulada + prediccion[, "BAJA+2"]]

    umbral_corte <- (1 / 40) * arbolito
    #stopifnot(nrow(dfuture) == length(prob_baja2))
    prob_baja2 <- tb_prediccion[, Predicted := as.numeric(prob_acumulada > umbral_corte)]

    }


    stopifnot(nrow(dataset[fold == fold_test, ]) == nrow(tb_prediccion))

    # Agrego calculo la ganancia
    ganancia_testing <- dataset[fold == fold_test][
      prob_baja2 > 1 / 40,
      sum(ifelse(clase_ternaria == "BAJA+2",
        117000, -3000
      ))
    ]

    message(ganancia_testing, " ")

    return(ganancia_testing)



}

In [412]:
#ArbolesCrossValidation <- function(x) {
  qfolds = 5
  semilla= PARAM$semilla_primigenia
  pagrupa= "clase_ternaria"

  #GLOBAL_iteracion <<- GLOBAL_iteracion + 1

  # generalmente  c(1, 1, 1, 1, 1 )  cinco unos
  divi <- rep(1, qfolds)

  # particiono en dataset en folds
  particionar(dataset, divi, seed= semilla, agrupa= pagrupa)

  ganancias <- mcmapply(EstimarGanancia,
    seq(qfolds), # 1 2 3 4 5
    MoreArgs= list(PARAM$rpart),
    SIMPLIFY= FALSE,
    mc.cores= detectCores()
  )

  dataset[, fold := NULL]

  # devuelvo la primer ganancia y el promedio
  # promedio las ganancias
  ganancia_promedio <- mean(unlist(ganancias))
  # aqui normalizo la ganancia
  ganancia_promedio_normalizada <- ganancia_promedio * qfolds

  message(ganancia_promedio_normalizada, " ")
#}

Warning message in mclapply(seq_len(n), do_one, mc.preschedule = mc.preschedule, :
“all scheduled cores encountered errors in user code”
Warning message in mean.default(unlist(ganancias)):
“argument is not numeric or logical: returning NA”
NA 



In [395]:
archivo_log <- "BO_log.txt"
archivo_BO <- "bayesian.RDATA"

# leo si ya existe el log
#  para retomar en caso que se se corte el programa
GLOBAL_iteracion <- 0
GLOBAL_mejor <- -Inf

if (file.exists(archivo_log)) {
  tabla_log <- fread(archivo_log)
  GLOBAL_iteracion <- nrow(tabla_log)
  GLOBAL_mejor <- tabla_log[, max(ganancia)]
}

In [396]:
# Defino la  Optimizacion Bayesiana

# cantidad de iteraciones de la Optimizacion Bayesiana
PARAM <- list()

PARAM$semilla_primigenia <- 100183
PARAM$experimento <- "HT312"

PARAM$BO_iter <- 2 #cantidad de iteraciones de la Bayesian Optimization

# la letra L al final de 1L significa ENTERO
PARAM$hs <- makeParamSet(
    makeNumericParam("cp", lower= -1, upper=-1),
    makeIntegerParam("minsplit", lower= 1, upper= 50),
    makeIntegerParam("minbucket", lower= 1, upper= 20),
    makeIntegerParam("maxdepth", lower= 3, upper= 5),
    forbidden= quote(minbucket > 0.5 * minsplit)
)
# minbuket NO PUEDE ser mayor que la mitad de minsplit

In [397]:
# carpeta de trabajo
setwd("/content/buckets/b1/exp")
dir.create(PARAM$experimento, showWarnings=FALSE)
setwd( paste0("/content/buckets/b1/exp/", PARAM$experimento ))

In [398]:
# Aqui comienza la configuracion de la Bayesian Optimization

funcion_optimizar <- ArbolesCrossValidation

configureMlr(show.learner.output= FALSE)

# configuro la busqueda bayesiana,
#  los hiperparametros que se van a optimizar
# por favor, no desesperarse por lo complejo
# minimize= FALSE estoy Maximizando la ganancia
obj.fun <- makeSingleObjectiveFunction(
  fn= funcion_optimizar,
  minimize= FALSE,
  noisy= TRUE,
  par.set= PARAM$hs,
  has.simple.signature= FALSE
)

ctrl <- makeMBOControl(
  save.on.disk.at.time= 600,
  save.file.path= archivo_BO
)

ctrl <- setMBOControlTermination(ctrl, iters= PARAM$BO_iter)
ctrl <- setMBOControlInfill(ctrl, crit= makeMBOInfillCritEI())

surr.km <- makeLearner("regr.km",
  predict.type= "se",
  covtype= "matern3_2", control= list(trace= TRUE)
)

In [399]:
ctrl <- makeMBOControl()
ctrl <- setMBOControlTermination(ctrl, iters = 100)
ctrl <- setMBOControlInfill(ctrl, crit = makeMBOInfillCritEI())

# Guarda automáticamente cada iteración
ctrl$save.on.disk.at.time <- 60   # guarda cada 60 segundos
ctrl$save.file.path <- archivo_BO # archivo .RData donde se guarda el estado

# 4) Lanzar / reanudar
if (!file.exists(archivo_BO)) {
    bayesiana_salida <- mbo(
    fun = obj.fun,
    learner = surr.km,
    control = ctrl
  )
} else {
  bayesiana_salida <- mboContinue(archivo_BO)
}

Computing y column(s) for design. Not provided.

Warning message in mclapply(seq_len(n), do_one, mc.preschedule = mc.preschedule, :
“all scheduled cores encountered errors in user code”
Warning message in mean.default(unlist(ganancias)):
“argument is not numeric or logical: returning NA”
Warning message in mclapply(seq_len(n), do_one, mc.preschedule = mc.preschedule, :
“all scheduled cores encountered errors in user code”
Warning message in mean.default(unlist(ganancias)):
“argument is not numeric or logical: returning NA”
Warning message in mclapply(seq_len(n), do_one, mc.preschedule = mc.preschedule, :
“all scheduled cores encountered errors in user code”
Warning message in mean.default(unlist(ganancias)):
“argument is not numeric or logical: returning NA”
Warning message in mclapply(seq_len(n), do_one, mc.preschedule = mc.preschedule, :
“all scheduled cores encountered errors in user code”
Warning message in mean.default(unlist(ganancias)):
“argument is not numeric or logical: retur

ERROR: Error in evalTargetFun.OptState(opt.state, xs, extras): Objective function output must be a numeric of length 1, but we got: NA


In [146]:
# inicio la optimizacion bayesiana
if (!file.exists(archivo_BO)) {
  bayesiana_salida <- mbo(
    fun= obj.fun,
    learner= surr.km,
    control= ctrl
  )
} else {
  bayesiana_salida <- mboContinue(archivo_BO)
}
# retomo en caso que ya exista

Computing y column(s) for design. Not provided.

[mbo] 0: cp=-1; minsplit=7962; minbucket=1189; maxdepth=9 : y = 0 : 0.0 secs : initdesign

[mbo] 0: cp=-1; minsplit=3666; minbucket=1360; maxdepth=15 : y = 0 : 0.0 secs : initdesign

[mbo] 0: cp=-1; minsplit=4076; minbucket=1704; maxdepth=4 : y = 0 : 0.0 secs : initdesign

[mbo] 0: cp=-1; minsplit=5951; minbucket=289; maxdepth=16 : y = 0 : 0.0 secs : initdesign

[mbo] 0: cp=-1; minsplit=4777; minbucket=1950; maxdepth=6 : y = 0 : 0.0 secs : initdesign

[mbo] 0: cp=-1; minsplit=7275; minbucket=162; maxdepth=6 : y = 0 : 0.0 secs : initdesign

[mbo] 0: cp=-1; minsplit=6141; minbucket=2750; maxdepth=11 : y = 0 : 0.0 secs : initdesign

[mbo] 0: cp=-1; minsplit=6426; minbucket=2059; maxdepth=19 : y = 0 : 0.0 secs : initdesign

[mbo] 0: cp=-1; minsplit=7505; minbucket=2944; maxdepth=8 : y = 0 : 0.0 secs : initdesign

[mbo] 0: cp=-1; minsplit=6001; minbucket=987; maxdepth=14 : y = 0 : 0.0 secs : initdesign

[mbo] 0: cp=-1; minsplit=1143; minbucke

ERROR: Error in backsolve(t(T), c.newdata, upper.tri = FALSE): singular matrix in 'backsolve'. First zero in diagonal [1]


In [ ]:
# almaceno los resultados de la Bayesian Optimization
# y capturo los mejores hiperparametros encontrados

tb_bayesiana <- as.data.table(bayesiana_salida$opt.path)

# ordeno en forma descendente por AUC = y
setorder(tb_bayesiana, -y)

# grabo para eventualmente poder utilizarlos en OTRA corrida
fwrite( tb_bayesiana,
  file= "BO_log.txt",
  sep= "\t"
)

# los mejores hiperparámetros son los que quedaron en el registro 1 de la tabla
PARAM$out$lgbm$mejores_hiperparametros <- tb_bayesiana[
  1, # el primero es el de mejor AUC
  list(cp, minsplit, minbucket, maxdepth)
]

print(PARAM$out$lgbm$mejores_hiperparametros)

In [ ]:
format(Sys.time(), "%a %b %d %X %Y")



---

